In [1]:
%matplotlib inline
import sys
sys.path.append("src") # add the source dir

In [2]:
import matplotlib.pyplot as plt
import os
from src.pipeline import Pipeline, MultiPipeline
from mne_bids import BIDSPath
import mne

## We will do an individual Analysis of subject 30, afterthat, we will statistically check all the other subject

In [3]:
bids_root = os.path.join('data', 'P3')
bids_path = BIDSPath(subject='039', session='P3', task='P3',datatype='eeg', suffix='eeg', root=bids_root)
pipeline = Pipeline(bids_path)

In [ ]:
from preprocessing import CleaningData, SimpleMNEFilter, PrecomputedICA

pipeline.load_data()
pipeline.set_montage()

### We will use our pipeline to perform following operations
- Loading bad segments, bad channels
- Apply filtering
- Loading Precomputed ICA

In [ ]:
pipeline.make_pipeline([CleaningData(bids_path), SimpleMNEFilter(0.5, 50, 'firwin'), PrecomputedICA(bids_path)])

In [ ]:
pipeline.set_custom_events_mapping(task='P3')
pipeline.apply_rereferencing('average')
pipeline.apply_resampling(512)

# ERP Peak Analysis

In [ ]:
from erpanalysis import ERPAnalysis
tmin = -0.1
tmax = 0.8
baseline = (None, 0)


## We will create epochs from custom mappings, where,

- Stimulus: A, B, C, D, E

- Conditions: Rare, Frequent

In [ ]:

erp = ERPAnalysis(tmin, tmax, baseline=baseline, reject_by_annotation=True)
# erp.compute_epochs(pipeline.raw, pipeline.events, pipeline.event_ids, baseline=baseline, reject_by_annotation=True)
pipeline.compute_epochs(erp)

## Analysis Epochs

In [ ]:
_, axs = plt.subplots(nrows=1, ncols=5, figsize=(8, 4));
erp.epochs['rare'].plot_psd_topomap(axes = axs);
_, axs = plt.subplots(nrows=1, ncols=5, figsize=(8, 4));
erp.epochs['freq'].plot_psd_topomap(axes = axs);

In [ ]:
erp.epochs['rare'].plot_image(picks=['Cz'], combine='mean');
erp.epochs['freq'].plot_image(picks=['Cz'], combine='mean');

In [ ]:

erp.epochs['A'].plot_image(picks=['Cz'], combine='mean');

## Analysis Evokeds

In [ ]:
epochs_dirty = erp.compute_epochs(pipeline.raw, pipeline.events, pipeline.event_ids, set_default=False, reject_by_annotation=False)
epochs_clean= erp.epochs.copy()
mne.viz.plot_compare_evokeds({'clean': epochs_clean['stimulus'].average(),'dirty':epochs_dirty['stimulus'].average()},picks="Cz")

In [ ]:
mne.viz.plot_compare_evokeds({'clean': epochs_clean['A'].average(),'dirty':epochs_dirty['A'].average()},picks="Cz")

In [ ]:
mne.viz.plot_compare_evokeds({'rare': epochs_clean['rare'].average(),'freq':epochs_clean['freq'].average()},picks="Cz")

## It seems it is futile to check for condition on each subject since, the number of events in epochs drop downs to a range between 0,5. There is a possiblity that we might be getting futile results

In [ ]:
_, axs = plt.subplots(nrows=1, ncols=1, figsize=(10, 4));
mne.viz.plot_compare_evokeds({'rare': epochs_clean['rare/D'].average(),'freq':epochs_clean['freq/D'].average()},axes=axs, picks="Cz")
# mne.viz.plot_compare_evokeds({'rare': epochs_clean['rare/B'].average(),'freq':epochs_clean['freq/B'].average()},axes=ax[1], picks="Cz")

In [ ]:
pipelines = MultiPipeline(bids_root)
erp = ERPAnalysis(tmin, tmax, baseline=(None, 0), reject_by_annotation=True, all_subjects=True)
pipelines.start_erp_analysis(erp);

In [ ]:
len(erp.epochs)

In [ ]:
def plt_all_subjects(_epochs, ax, sub_id):
    mne.viz.plot_compare_evokeds({'rare': _epochs['rare'].average(),'freq':_epochs['freq'].average()},axes=ax, title='Subject {}'.format(sub_id), picks="Cz")

In [ ]:
mne.viz.plot_compare_evokeds({'rare': erp.epochs[38]['rare'].average(),'freq':erp.epochs[38]['freq'].average()},picks="Cz")
mne.viz.plot_compare_evokeds({'rare': epochs_clean['rare'].average(),'freq':epochs_clean['freq'].average()},picks="Cz")

In [ ]:
evokeds = [epochs_clean[name].average() for name in ('rare', 'freq')]

In [ ]:
colors = 'red', 'green'
title = 'MNE sample data\nleft vs right (A/V combined)'
mne.viz.plot_evoked_topo(evokeds, color=colors, title=title, background_color='w')
plt.show()

In [ ]:
# %matplotlib qt 
# _, axs = plt.subplots(nrows=2, ncols=4, sharex=False, sharey=False);
# ax = axs.ravel()
# dfl = erp.epochs[:8]
# for i, ep in enumerate(dfl):
#     ax[i].set_title('subject {}'.format(i))
#     plt_all_subjects(ep, ax[i], i+1)
# plt.tight_layout()

In [ ]:
import numpy as np
blocks = np.array([ list(range(10*x + 1, 10*x + 6)) for x in range(1, 6)])
rare = np.array([ x + i for i, x in enumerate(range(11, 56, 10))]).tolist()
freq = np.setdiff1d(blocks.flatten(), rare).tolist()
stimlus = ['A', 'B', 'C', 'D', 'E']
evts_stim = pipeline.event_ids
rare = [k for k in evts_stim.keys() if k.split('/')[1] == 'rare']
freq = [k for k in evts_stim.keys() if k.split('/')[1] == 'freq']
rare_dict = dict((k, pipeline.event_ids[k]) for k in rare)
freq_dict = dict((k, pipeline.event_ids[k]) for k in freq)

rare_epochs = mne.Epochs(pipeline.raw,pipeline.events, rare_dict,tmin=-0.2,tmax=0.8, reject_by_annotation=False).copy()
freq_epochs = mne.Epochs(pipeline.raw,pipeline.events, freq_dict,tmin=-0.2,tmax=0.8, reject_by_annotation=False).copy()
mne.viz.plot_compare_evokeds({'rare': rare_epochs.average(), 'freq': freq_epochs.average()}, picks='Cz')

In [ ]:
mne.viz.plot_compare_evokeds({'rare': epochs_clean['rare'].average(),'freq':epochs_clean['freq'].average()},picks="Cz")

# ERP Analysis